In [17]:
from google.cloud import storage
import os
import pandas as pd 
import datalab.storage as gcs
from google.cloud import bigquery
import sys 

API_KEY_PATH = "/Users/jideofor/Documents/cs397/Patents-Research-abd8b4aaf0a8.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = API_KEY_PATH
PROJECT_ID = "patents-research-275923"
BUCKET_NAME = "patents-research"

def upload_df_toGoogle(data_frame):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"
    
    # create bucket if it does not exists
    if not STORAGE_CLIENT.lookup_bucket(BUCKET_NAME):
        STORAGE_CLIENT.create_bucket(BUCKET_NAME, project=PROJECT_ID)

    # upload to google cloud
    storage_client = storage.Client()
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob("patent_research/data.tsv")

    blob.upload_from_string(data_frame.to_csv(sep='\t'),'text/tab-separated-values')

    print("File data_frame uploaded to patent_research/data.tsv")

## Query to get majority of data

In [ ]:
QUERY = """
SELECT app.patent_id as patent_id, app.date as publication_date, filterData.bkwdCitations_5, filterData.fwrdCitations_5, patent.title, patent.abstract as abstract_text, summary.text as summary_text
FROM
  `patents-public-data.patentsview.brf_sum_text` as summary,
  `patents-public-data.patentsview.patent` as patent,
  `patents-public-data.patentsview.application` as app,
  (SELECT DISTINCT cpc.patent_id, IFNULL(citation_5.bkwdCitations_5, 0) as bkwdCitations_5, IFNULL(citation_5.fwrdCitations_5, 0) as fwrdCitations_5
  FROM
    `patents-public-data.patentsview.cpc_current` AS cpc
    LEFT JOIN
    (SELECT  b.patent_id, b.bkwdCitations_5, f.fwrdCitations_5
      FROM 

        (SELECT 
          cited.patent_id,
          COUNT(*) as fwrdCitations_5
          FROM 
          `patents-public-data.patentsview.uspatentcitation` AS cited,
          `patents-public-data.patentsview.application` AS apps
        WHERE
          apps.country = 'US'
          AND cited.patent_id = apps.patent_id 
          AND cited.date >= apps.date AND SAFE_CAST(cited.date AS DATE) <= DATE_ADD(SAFE_CAST(apps.date AS DATE), INTERVAL 5 YEAR) -- get in 5year interval 
         GROUP BY 
         cited.patent_id) AS f,

       (SELECT 
          cited.patent_id,
          COUNT(*) as bkwdCitations_5
          FROM 
          `patents-public-data.patentsview.uspatentcitation` AS cited,
          `patents-public-data.patentsview.application` AS apps
        WHERE
          apps.country = 'US'
          AND cited.patent_id = apps.patent_id 
          AND cited.date < apps.date AND SAFE_CAST(cited.date AS DATE) >= DATE_SUB(SAFE_CAST(apps.date AS DATE), INTERVAL 5 YEAR) -- get in 5year interval 
         GROUP BY 
         cited.patent_id) AS b
      WHERE
      b.patent_id = f.patent_id AND b.bkwdCitations_5 IS NOT NULL AND f.fwrdCitations_5 IS NOT NULL) AS citation_5 
      ON cpc.patent_id=citation_5.patent_id
      WHERE
       (cpc.subsection_id IN ('C05', 'C07', 'C08', 'C09', 'C11', 'C12', 'C13', 'C25', 'C40')
        OR cpc.group_id in ('A01G', 'A01H', 'A61K', 'A61P', 'A61Q', 'B01F', 'B01J', 'B81B', 'B82B', 'B82Y','G01N', 'G16H')))
  as filterData
  WHERE
  app.patent_id = filterData.patent_id AND summary.patent_id = app.patent_id AND app.patent_id = patent.id 
"""

## Query to get claims data

In [18]:
QUERY_CLAIMS = """
SELECT filteredIDs.patent_id AS patent_id, CAST(claim.sequence as INT64)  as claim_number, claim.text as claim_text
FROM
  `patents-public-data.patentsview.claim` AS claim,
  (SELECT DISTINCT patent.id as patent_id
  FROM
    `patents-public-data.patentsview.patent` as patent,
    `patents-public-data.patentsview.cpc_current` AS cpc
  WHERE
    patent.id=cpc.patent_id
    AND (cpc.subsection_id IN ('C05', 'C07', 'C08', 'C09', 'C11', 'C12', 'C13', 'C25', 'C40')
          OR cpc.group_id in ('A01G', 'A01H', 'A61K', 'A61P', 'A61Q', 'B01F', 'B01J', 'B81B', 'B82B', 'B82Y','G01N', 'G16H'))) AS filteredIDs
WHERE
  filteredIDs.patent_id = claim.patent_id
ORDER BY
  patent_id,
  claim_number
"""

In [20]:
df_claims = pd.read_gbq(QUERY_CLAIMS, project_id=PROJECT_ID, dialect='standard', progress_bar_type='tqdm')

Downloading: 100%|██████████| 17144505/17144505 [19:36<00:00, 14568.01rows/s]


In [28]:
df_claims = df_claims.set_index('patent_id')

In [64]:
#https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/
def get_claim_blob(df,id):
    base = df.loc[[id]]
    text = ""
    for _,row in base.iterrows():
        start = "<start -claim{}>".format(row['claim_number'])
        end = "<end -claim{}>".format(row['claim_number'])
        text += "{}{}{}".format(start,row['claim_text'], end)
    return text
        
def get_claim_df(base_df):
    
    claim_df = pd.DataFrame(columns=['patent_id', 'claims'])
    patentIds = set(base_df.index)
    n = len(patentIds)
    
    i=0
    for id in patentIds:
        claim_df.loc[i]=[id,get_claim_blob(base_df,id)]
        i+=1
        print("Progress {}/{}".format(i,n), end="\r", flush=True)
    return claim_df


In [ ]:
merged_claims_df = get_claim_df(df_claims)

In [ ]:
# save just in case
df.to_pickle("~/Desktop/claim.pickle",compression="gzip")  # where to save it, usually as a .pkl
# save to bigquery
pd.to_gbq(merged_claims_df, "cleaned_claims", project_id =PROJECT_ID)

## Get the main db

In [58]:
df_main = pd.read_gbq(QUERY, project_id=PROJECT_ID, dialect='standard', progress_bar_type='tqdm')

## Merge  the database

In [ ]:
# Merge the two db's
full_df = df_claims.set_index('patent_id').join(merged_claims_df, how='inner') 
# Upload to GoogleCloud
upload_df_toGoogle(full_df)

